# ALGORYTM CARLIER

1. Pierwsza czescia zadania byla implmentacja algorytmu Carlier, ktory dla wszystkich instancji poda poprawny wynik.

In [3]:
import copy
from datareader import get_data
from makespan import makespan, get_order
from schrage import schrage_n2, schrage_n2_pmtn
from timeit import default_timer as timer

Pobieranie listy instancji:

In [4]:
task_list = ["data.000", "data.001", "data.002", "data.003", "data.004", "data.005", "data.006", "data.007", "data.008"]


Sama implementacja podstawowego algorytmu Carlier wyglada nastepujaco (typ WIDE LEFT):

In [6]:
# wide left
def carlier_wl(tasks):
    # ----- "GLOBAL" VARIABLES DECLARATION ----- #
    # u: cmax for schrage
    # pi: tasks sorted with schrage
    # ub: upper border
    # lb: lower border

    global u
    global pi
    global ub
    global lb
    global pi_list

    u, pi = schrage(copy.deepcopy(pi_list[0]))
    if u < ub:
        ub = u

    # find b, a, c values
    b = calculate_b(u, pi)
    a = calculate_a(b, u, pi)
    c = calculate_c(b, a, pi)

    #  if c doesn't exist
    if c < 0:
        return ub

    # calculate r1, p1, q1
    r_1 = pi[c + 1].times[0]
    p_1 = 0
    q_1 = pi[c + 1].times[2]

    for i in range(c + 1, b + 1):
        p_1 += pi[i].times[1]
        if r_1 > pi[i].times[0]:
            r_1 = pi[i].times[0]
        if q_1 > pi[i].times[2]:
            q_1 = pi[i].times[2]

    # LEFT NODE
    r_pi_c = pi[c].times[0]
    pi[c].times[0] = max(pi[c].times[0], r_1 + p_1)

    p_2 = 0
    r_2 = pi[c].times[0]
    q_2 = pi[c].times[2]

    for i in range(c, b + 1):
        p_2 += pi[i].times[1]
        if r_2 > pi[i].times[0]:
            r_2 = pi[i].times[0]
        if q_2 > pi[i].times[2]:
            q_2 = pi[i].times[2]

    lb, ptmn_order = schrage_pmtn(copy.deepcopy(pi))
    lb = max(max(r_1 + q_1 + p_1, r_2 + q_2 + p_2), lb)
    if lb < ub:
        pi_list.append(copy.deepcopy(pi))
    pi[c].times[0] = r_pi_c

    # RIGHT NODE
    r_pi_c = pi[c].times[2]
    pi[c].times[2] = max(pi[c].times[2], q_1 + p_1)

    p_2 = 0
    r_2 = pi[c].times[0]
    q_2 = pi[c].times[2]

    for i in range(c, b + 1):
        p_2 += pi[i].times[1]
        if r_2 > pi[i].times[0]:
            r_2 = pi[i].times[0]
        if q_2 > pi[i].times[2]:
            q_2 = pi[i].times[2]

    lb, order = schrage_pmtn(copy.deepcopy(pi))
    lb = max(max(r_1 + q_1 + p_1, r_2 + q_2 + p_2), lb)

    if lb < ub:
        pi_list.append(copy.deepcopy(pi))
    pi[c].times[2] = r_pi_c

    # remove checked node from the list
    pi_list.pop(0)
    return carlier_wl(tasks)

Porownano dzialanie algorytmu Carlier z algorytmem Schrage i Schrage PMTN

In [ ]:
from carlier import carlier_wl

for i in range(0, len(task_list)):
    tasks = get_data(task_list[i])
    #result = result_list[i]

    print("TEST: ", task_list[i])

    # SCHRAGE ORDER
    schrage_n2_order, schrage_n2_time = schrage_n2(tasks)
    schrage_n2_makespan = makespan(schrage_n2_order, tasks)

    # SCHRAGE PMTN ORDER
    schrage_n2_ptmn_makespan, schrage_n2_ptmn_order, schrage_n2_ptmn_time = schrage_n2_pmtn(tasks)

    # CARLIER ORDER
    carlier_makespan_pure, carlier_time_pure = carlier_wl(copy.deepcopy(tasks))

TEST:  data.000
-
[CARLIER WL] makespan: 228, time: 6.4338240000552105
RESULT:  OK  | SHOULD BE:  228
-
[CARLIER DL] makespan: 228, time: 5.846691999977338
RESULT:  OK  | SHOULD BE:  228
-
[CARLIER GREEDY] makespan: 228, time: 5.621679000000768
RESULT:  OK  | SHOULD BE:  228
---------------------------------------------
TEST:  data.001
-
[CARLIER WL] makespan: 2976, time: 1431.8746610000517
RESULT:  BAD RESULT  | SHOULD BE:  3026
-
[CARLIER DL] makespan: 3026, time: 23420.981168000028
RESULT:  OK  | SHOULD BE:  3026
-
